In [69]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import re
import json
import pickle

In [70]:
import warnings
warnings.filterwarnings("ignore")

In [71]:
df = pd.read_csv('pop.csv')
df.head()

,Country Name,Country Code,Series Name,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2024 [YR2024]
0,Afghanistan,AFG,"Population, total",8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,...,32758020.0,33736494.0,34656032.0,35530000.0,36373000.0,37209000.0,38055000.0,38914000.0,39779000.0,NaN
1,Albania,ALB,"Population, total",1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,...,2889104.0,2880703.0,2876101.0,2884000.0,2892000.0,2900000.0,2907000.0,2913000.0,2919000.0,NaN
2,Algeria,DZA,"Population, total",11124888.0,11404859.0,11690153.0,11985136.0,12295970.0,12626952.0,12980267.0,...,39113313.0,39871528.0,40606052.0,41318000.0,42008000.0,42679000.0,43333000.0,43969000.0,44584000.0,NaN
3,American Samoa,ASM,"Population, total",20013.0,20486.0,21117.0,21882.0,22698.0,23520.0,24321.0,...,55437.0,55537.0,55599.0,56000.0,56000.0,56000.0,56000.0,56000.0,56000.0,NaN
4,Andorra,AND,"Population, total",13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,...,79223.0,78014.0,77281.0,77000.0,77000.0,77000.0,77000.0,77000.0,77000.0,NaN


In [72]:
bd=df.loc[df['Country Name']=='India']
bd.drop(['Country Name','Country Code','Series Name'],axis=1,inplace=True)
bd = bd.T
bd.head()

,103
1960 [YR1960],449480608.0
1961 [YR1961],458494963.0
1962 [YR1962],467852537.0
1963 [YR1963],477527970.0
1964 [YR1964],487484535.0


In [73]:
bd.dropna(inplace=True)
bd=bd.reset_index().rename(columns={103:'population','index':'year'})
bd['year'] = bd['year'].str.replace(r'\[YR\d+\]', '', regex=True)
bd.head()

,year,population
0,1960,449480608.0
1,1961,458494963.0
2,1962,467852537.0
3,1963,477527970.0
4,1964,487484535.0


In [74]:
x = bd.iloc[:, 0].values.reshape(-1, 1)
y = bd.iloc[:, 1].values.reshape(-1, 1)
model = LinearRegression().fit(x, y)
y_pred = model.predict([[2019]])
y_pred

array([[1.36407221e+09]])

In [75]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import json

In [76]:
def country_list_gen(df):
    df.rename(columns={'Country Name': 'country_name'}, inplace=True)
    df['country_name'] = df['country_name'].apply(lambda row: row.lower())
    lists = df['country_name'].unique().tolist()
    with open('country_list.json', 'w', encoding='utf-8') as f:
        json.dump(lists, f, ensure_ascii=False, indent=4)
    return lists, df

In [77]:
# temp = pd.read_csv('pop.csv')
# df = df.loc[df['country_name'] == country]
# df.drop(['country_name', 'Country Code', 'Series Name'], axis=1, inplace=True)
# df.head()

In [78]:

# def selecting_country(df, country):
#     df = df.loc[df['country_name'] == country]
#     df.drop(['country_name', 'Country Code', 'Series Name'], axis=1, inplace=True)
#     df = df.T
#     df.columns = df.iloc[0]  # Set the first row as column names
#     df = df[1:]  # Skip the first row

#     # Extract only the numeric part from the year column
#     df.index = df.index.str.extract(r'(\d+)').astype(int)

#     df.dropna(inplace=True)
#     df = df.reset_index()
#     return df

# def prediction_model(df):
#     x = df.iloc[:, 0].values.reshape(-1, 1)
#     y = df.iloc[:, 1].values.reshape(-1, 1)
#     model = LinearRegression().fit(x, y)
#     return model

# def prediction(model, year):
#     return int(model.coef_[0][0] * year + model.intercept_[0])

def selecting_country(df, country):
    df = df.loc[df['Country Name'].str.lower() == country.lower()]
    df.drop(['Country Name', 'Country Code', 'Series Name'], axis=1, inplace=True)
    
    # Transpose the DataFrame
    df = df.T.reset_index()
    
    # Extract only the numeric part from the year column
    df.columns = ['Year', country]
    df['Year'] = df['Year'].str.extract(r'(\d+)').astype(int)

    df.dropna(inplace=True)
    return df

def prediction_model(df):
    x = df[['Year']].values
    y = df.iloc[:, 1].values.reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    return model

def prediction(model, year):
    return int(model.predict([[year]])[0, 0])

In [79]:
def main():
    country = input("Please input the country name: ").lower()
    year = int(input("Please input the year to predict: "))
    df = pd.read_csv('pop.csv')
    if country in df['Country Name'].str.lower().unique():
        df = selecting_country(df, country)
        model = prediction_model(df)
        result = prediction(model, year)
        
        
        save_model(model)

        print(f"\n Result: {country.upper()} population in {year} will be {result:,d}")
    else:
        print('Kindly check country name spelling from the available country list.')

if __name__ == "__main__":
    main()

Model saved as model.pkl

 Result: INDIA population in 2023 will be 1,429,937,114


In [80]:
def save_model(model, filename='model.pkl'):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved as {filename}")